In [16]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

Model classes and functions

In [34]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

class FC_Dataset(Dataset):
    def __init__(self, data_full):
        self.data = data_full[:,:-1]
        self.target = data_full[:, -1]
        self.nfeatures = self.data.shape[1]
        self.nclasses = len(torch.unique(self.target))
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]
    
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_1, hidden_2):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_1),
            nn.ReLU(),
            nn.Linear(hidden_1, hidden_2),
            nn.ReLU(),
            nn.Linear(hidden_2, 1)
        )
    
    def forward(self, x):
        logits = self.network(x)
        return logits
    
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y.view(-1, 1))  # Ensure y has the correct shape

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.view(-1, 1)).item()

    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

Data load and model object

In [42]:
data_full = pd.read_csv('model_input.csv')

data_tensor = torch.tensor(data_full.values.astype(np.float32)).to(device)
dataset = FC_Dataset(data_tensor)
train, test = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])
train_dataloader = DataLoader(train, batch_size=64)
test_dataloader = DataLoader(test, batch_size=64)

model = NeuralNetwork(dataset.nfeatures, 20, 5).to(device)

# logits = model(dataset.data[0])
# pred_probab = nn.Softmax(dim=0)(logits)
# y_pred = pred_probab.argmax()
# print(f"Predicted class: {y_pred}")

Model training

In [43]:
learning_rate = 1e-3
batch_size = 32
epochs = 250

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Training complete")

Epoch 1
-------------------------------
loss: 5423.367188  [    0/ 3408]
Test Error: 
 Avg loss: 48501.873047 

Epoch 2
-------------------------------
loss: 48717.371094  [    0/ 3408]
Test Error: 
 Avg loss: 39081.634208 

Epoch 3
-------------------------------
loss: 39274.996094  [    0/ 3408]
Test Error: 
 Avg loss: 31492.239955 

Epoch 4
-------------------------------
loss: 31665.738281  [    0/ 3408]
Test Error: 
 Avg loss: 25377.829381 

Epoch 5
-------------------------------
loss: 25533.498047  [    0/ 3408]
Test Error: 
 Avg loss: 20451.677455 

Epoch 6
-------------------------------
loss: 20591.343750  [    0/ 3408]
Test Error: 
 Avg loss: 16482.783761 

Epoch 7
-------------------------------
loss: 16608.089844  [    0/ 3408]
Test Error: 
 Avg loss: 13285.095912 

Epoch 8
-------------------------------
loss: 13397.508789  [    0/ 3408]
Test Error: 
 Avg loss: 10708.709891 

Epoch 9
-------------------------------
loss: 10809.553711  [    0/ 3408]
Test Error: 
 Avg loss: